In [0]:
from google.colab import drive
import matplotlib.pyplot as plt

In [36]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Bigquery
## Load the Data for 4 CICS data from Bigquery

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
'''
%%bigquery --project ds-student-data df
SELECT *
FROM `ds-student-data.Distinctive.ChronicAbsence` 
'''

In [0]:
#Getting the ChronicAbsence data
#Change the last where clause by condition

%%bigquery --project ds-student-data df

SELECT a.NameOfInstitution,a.StudentUniqueId, sde.GradeLevel, a.LastSurname, a.FirstName, a.NumberOfTotalAbsences, b.AverageAbsence
FROM `ds-student-data.StarterPack.StudentAttendance` as a
INNER JOIN
(SELECT ROUND(AVG(NumberOfTotalAbsences),1) AverageAbsence, NameOfInstitution
FROM `ds-student-data.StarterPack.StudentAttendance`
GROUP BY NameOfInstitution ) as b
ON a.NameOfInstitution = b.NameOfInstitution
LEFT JOIN `ds-student-data.StarterPack.StudentDemographicsAndEnrollment` as sde
ON a.StudentUniqueID=sde.StudentUniqueID
WHERE a.NumberOfTotalAbsences >=10

In [0]:
#df.shape

(551, 7)

In [0]:
!ls /content/gdrive/My\ Drive/Chronic_absenteeism_letter/

 2020-01-23				       'Chronic Absence.xlsx'
'Chronic Absence - Absence List Import.csv'     Chronic_Absenteeism.ipynb
'Chronic Absence Letter #1 - Jan '\''20.gdoc'   test


In [0]:
schoolname = {'Prairie':'CICS Prairie', 'Bucktown':'CICS Bucktown', 'Distinctive College Prep Harper Woods':'DCP Harper Woods',
       'Distinctive College Prep Redford':'DCP Redford', 'West Belden':'CICS West Belden', 'Irving Park':'CICS Irving Park'}

In [0]:
df['NameOfInstitution'] = df['NameOfInstitution'].map(schoolname)

# Load from CSV

##use this code to load data is not on bigquery, AIM & PEC

In [0]:
#Data saved in csv/excel
import pandas as pd
import glob, os

data_dir= '/content/gdrive/My Drive/Chronic_absenteeism_letter/'
filen = '2.13ChronicAbsence.csv'

os.chdir(data_dir)

df=pd.read_csv(filen)

print("Column headings:")
print(df.columns)


Column headings:
Index(['NameOfInstitution', 'StudentUniqueId', 'GradeLevel', 'LastSurname',
       'FirstName', 'NumberOfTotalAbsences', 'AverageAbsence'],
      dtype='object')


In [0]:
df.shape

(174, 7)

In [0]:
df.tail()

,NameOfInstitution,StudentUniqueId,GradeLevel,LastSurname,FirstName,NumberOfTotalAbsences,AverageAbsence
169,Art in Motion,50235763,8,Noel,Rashaud,14,4
170,Art in Motion,50189119,7,Prince,Toney,19,4
171,Art in Motion,50300256,7,Riley,Dinai,10,4
172,Art in Motion,50251714,7,Smith,Jalyn,15,4
173,Art in Motion,50158879,8,Smith,Tajah,10,4


#Getting the Images  & Principal info

In [0]:
import pandas as pd
import glob, os

data_dir= '/content/gdrive/My Drive/Chronic_absenteeism_letter/'
filename = 'Chronic Absence.xlsx'

os.chdir(data_dir)


In [0]:
dname=pd.read_excel(filename, sheet_name='Images  & PrincipalTable')

In [0]:
df.NameOfInstitution.unique()

array(['Plymouth Educational Center', 'Art in Motion'], dtype=object)

In [0]:
dc=df.groupby("NameOfInstitution")['AverageAbsence'].mean().round(1).to_dict()

In [0]:
dc

{'Art in Motion': 4, 'Plymouth Educational Center': 8}

In [0]:
#Using whole DS average for PEC
dc['Plymouth Educational Center']= 6.1

In [0]:
#Getting the total number of absence for all schools
%%bigquery --project ds-student-data mni

SELECT AVG(NumberOfTotalAbsences) as avgDS
FROM `ds-student-data.StarterPack.StudentAttendance`

In [0]:
#Using whole DS average for DCP 
dc['DCP Harper Woods']=mni.avgDS.loc[0].round(1)
dc['DCP Redford']=mni.avgDS.loc[0].round(1)

In [0]:
mni.avgDS.loc[0].round(1)

6.1

In [0]:
dc

{'Art in Motion': 4, 'Plymouth Educational Center': 6.1}

In [0]:
df['AverageAbsence'] = df['NameOfInstitution'].map(dc)

In [0]:
df.head()

,NameOfInstitution,StudentUniqueId,GradeLevel,LastSurname,FirstName,NumberOfTotalAbsences,AverageAbsence
0,Plymouth Educational Center,20007123,K,Bridges,Walter,20,6.1
1,Plymouth Educational Center,20007170,K,Brown,AlayJia,63,6.1
2,Plymouth Educational Center,20007124,K,Cabbil,Heaven,13,6.1
3,Plymouth Educational Center,20007279,K,Caldwell,Liam,10,6.1
4,Plymouth Educational Center,20007197,K,Clark,Chiron,18,6.1


In [0]:
links = dict(zip(dname.Campus, dname.Link))

In [0]:
links['Art in Motion']='https://www.distinctiveschools.org/uploads/8/4/9/1/84911918/published/aim-circle-logo-color-withwhite.png?1581611169'

In [0]:
links

{'Art in Motion': 'https://www.distinctiveschools.org/uploads/8/4/9/1/84911918/published/aim-circle-logo-color-withwhite.png?1581611169',
 'CICS Bucktown': 'https://www.cicsbucktown.org/uploads/8/4/9/1/84911918/published/cics-bucktown-horz.png?1531843975',
 'CICS Irving Park': 'https://www.cicsirvingpark.org/uploads/8/4/9/1/84911918/published/cics-irving-park-horz.png?1531844890',
 'CICS Prairie': 'https://www.cicsprairie.org/uploads/8/4/9/1/84911918/published/cics-prairie-horz.png?1531844962',
 'CICS West Belden': 'https://www.cicswestbelden.org/uploads/8/4/9/1/84911918/published/cics-west-belden-horz.png?1531845057',
 'DCP Harper Woods': 'https://www.dcpharperwoods.org/uploads/8/4/9/1/84911918/dscp-stacked-horizontal.png',
 'DCP Redford': 'https://www.dcpharperwoods.org/uploads/8/4/9/1/84911918/dscp-stacked-horizontal.png',
 'Plymouth Educational Center': 'https://www.plymouthed.org/uploads/8/4/9/1/84911918/published/pec-navylogo_1.png?1561052576'}

In [0]:
#Make a dictionary matching the campus to Director names
dnames = dict(zip(dname.Campus, dname.Director))

In [0]:
dnames

{'Art in Motion': 'Ms. Kara May',
 'CICS Bucktown': "Ms. Sarah O'Connell",
 'CICS Irving Park': 'Mr. JW Kuebler',
 'CICS Prairie': 'Ms. Jennifer Harth',
 'CICS West Belden': 'Ms. Colleen Collins',
 'DCP Harper Woods': 'Ms. Cassie Williams',
 'DCP Redford': 'Ms. Camille Hibbler',
 'Plymouth Educational Center': 'Ms. Schranda Battle'}

In [0]:
mails = {'CICS Bucktown':'soconnell@distinctiveschools.org','CICS Irving Park': 'jkuebler@distinctiveschools.org',
 'CICS Prairie': 'jharth@distinctiveschools.org',
 'CICS West Belden': 'ccollins@distinctiveschools.org',
 'DCP Harper Woods': 'cwilliams@distinctiveschools.org',
 'DCP Redford': 'chibbler@distinctiveschools.org' ,
 'Art in Motion': 'kmay@distinctiveschools.org',
 'Plymouth Educational Center': 'sbattle@distinctiveschools.org' }

# Create the letter

In [37]:
!pip3 install reportlab

In [0]:
import os

def createFolder(directory):

    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        



In [39]:
maxnumber = df['NumberOfTotalAbsences'].max()
maxnumber

63

In [0]:
from reportlab.lib import colors
from reportlab.graphics.shapes import Drawing, String
from reportlab.graphics.charts.barcharts import HorizontalBarChart


def make_drawing(row):
    drawing = Drawing()
    data = [(row['NumberOfTotalAbsences'], row['AverageAbsence']),]
    names = [row['FirstName'], 'Classmates\n /Compañeros']
    
    bc = HorizontalBarChart()
    bc.x = 20
    bc.y = 50
    bc.height = 150
    bc.width = 400
    bc.data = data
    bc.strokeColor = colors.black
    bc.valueAxis.valueMin = 0
    bc.valueAxis.visibleGrid = True
    bc.valueAxis.valueMax = maxnumber
    bc.valueAxis.valueStep = 10

    bc.barLabels.fontName = "Helvetica"
    bc.barLabels.fontSize = 12
    bc.barLabelFormat = '%d'
    bc.barLabels.nudge = 7
    
    bc.categoryAxis.labels.boxAnchor = 'ne'
    bc.categoryAxis.labels.dx = -10
    bc.categoryAxis.labels.dy = 0 
    bc.categoryAxis.labels.fontName = 'Helvetica'
    bc.categoryAxis.categoryNames = names
    
    bc.valueAxis.visible = True
    #Customize the color of the bars  
    bc.bars[(0, 0)].fillColor = colors.steelblue 
    bc.bars[(0, 1)].fillColor = colors.lightsteelblue
    drawing.add(bc)

    return drawing

In [0]:
import datetime
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import PageBreak
from reportlab.lib.utils import ImageReader
 
def build_a_letter(row, test=True):
    '''
    Input: dataframe row
    Output: pdf file named after studentID
    '''
    #I name the pdf name after the Initals and StudentID
  
    name = str(row['FirstName'][0])+ str(row['LastSurname'][0])+str(row['StudentUniqueId'])+'.pdf'
    full_name = row['FirstName']+' ' + row['LastSurname']
    school_name = row.NameOfInstitution
    grade = str(row.GradeLevel)
    formatted_time = datetime.date.today().isoformat()
    
    if test:
      outfiledir = data_dir+ 'test'+'/'+str(school_name)+'/'+grade+'/'
    else:
      outfiledir = data_dir+ str(formatted_time)+'/'+str(school_name)+'/'+grade+'/'
    createFolder(outfiledir)
    outfilepath = os.path.join(outfiledir, name)
    #print(outfilepath)

    doc = SimpleDocTemplate(outfilepath, pagesize=letter,
                        rightMargin=72,leftMargin=72,
                        topMargin=72,bottomMargin=18)
    Story=[]

    if school_name == 'DCP Harper Woods' or school_name =='DCP Redford':
      logo = Image(links[school_name], width=2*inch, height=0.42*inch, kind='proportional')
    else:
      logo = Image(links[school_name])

    
    NumberOfTotalAbsences = str(row.NumberOfTotalAbsences)
    story = []
    #Set the logo 
    Story.append(logo)
     
    styles=getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
    ptext = '<font size=11>%s</font>' % formatted_time
     
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 12))
     
    ptext = '<font size=11>Dear Family of %s: </font>' % (full_name)
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    
    ptext = '<font size=11>We are writing to you because <u>our records show \
             that your child has missed more than 10 % of the school year\
              to date.</u> Please understand that all absences are considered \
              unexcused until proper documentation is presented in accordance \
              with the school handbook.  Regular attendance at school is critical\
               for a child’s educational growth and achievement, and we are \
               committed to your child’s success.  We look forward to partnering\
                with you to ensure that your child continues to have a great \
                year of learning and growth.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=11>If absences continue, we will schedule a time for you\
     to come to school for a meeting with our administration, with the goal being\
      to identify the factors contributing to your child’s absenteeism, discuss \
      ways we can improve your child’s attendance this year, and commit to a plan\
       of action together. These steps will be taken in accordance with the \
       attendance policy for Distinctive Schools.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
     
    ptext = '<font size=11>Thank you for your attention to this matter, and \
    thank you in advance for your partnership and a dedication to ensuring that \
    your child is set up for success in school this year. I look forward to \
    working with you to support your child in having a positive and productive year. </font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    ptext = '<font size=11>Sincerely,</font>'
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 24))
    ptext = '<font size=11>%s</font>'% (dnames[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=11>Director, %s</font>'% (school_name)
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=11>%s</font>'% (mails[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(PageBreak()) 
    
    #Second page spanish version
    Story.append(logo)
     
    styles=getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
    ptext = '<font size=11>%s</font>' % formatted_time
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=11><u><b>Primera notificación</b></u></font>' 
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=11>Estimada familia de %s: </font>' % (full_name)
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext ='<font size=11>Le escribimos porque <u>nuestros registros muestran que \
    su hijo ha perdido más del 10 % del año escolar hasta la fecha.</u> Por favor, \
    comprenda que todas las ausencias se consideran injustificadas hasta que se \
    presente la documentación adecuada de acuerdo con el manual de la escuela. \
    La asistencia regular a la escuela es fundamental para el crecimiento y el \
    rendimiento educativo de un niño, y estamos comprometidos con el éxito de su\
     hijo. Esperamos asociarnos con usted este año para asegurar que su hijo \
     continúe teniendo un gran año de aprendizaje y crecimiento.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=11>Si las ausencias continúan, programaremos un horario \
    para que usted venga a la escuela a una reunión con nuestra administración \
    con el objetivo de identificar los factores que contribuyen al absentismo de\
     su hijo, discutir formas en que podemos mejorar la asistencia de su hijo \
     este año y comprometerse a un plan de acción juntos. Estos pasos se tomarán\
      de acuerdo con la política de asistencia de Distinctive Schools. </font>' 
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
     
    ptext = '<font size=11>Gracias por su atención a este asunto y gracias de \
    antemano por su colaboración y dedicación para garantizar que su hijo esté \
    preparado para el éxito en la escuela este año. Espero con interés trabajar \
    con usted para asegurarnos de que su hijo tenga un año positivo y productivo.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    ptext = '<font size=11>Atentamente,</font>'
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 24))
    ptext = '<font size=11>%s</font>'% (dnames[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=11>Director, %s</font>'% (school_name)
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=11>%s</font>'% (mails[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(PageBreak()) 

    #Third page barchart
    title_style = styles['Heading1']
    title_style.alignment = 1
    title = Paragraph("Absences Summary/Resumen de ausencias", title_style)
    Story.append(title)
    Story.append(Spacer(1, 24))
    pic = make_drawing(row)
    Story.append(pic)

    doc.build(Story)

In [0]:
def build_english_letter(row, test=True):
    '''
    Input: dataframe row
    Output: pdf file named after studentID
    '''
    #I name the pdf name after the Initals and StudentID
  
    name = str(row['FirstName'][0])+ str(row['LastSurname'][0])+str(row['StudentUniqueId'])+'.pdf'
    full_name = row['FirstName']+' ' + row['LastSurname']
    school_name = row.NameOfInstitution
    grade = str(row.GradeLevel)
    formatted_time = datetime.date.today().isoformat()
    
    if test:
      outfiledir = data_dir+ 'test'+'/'+str(school_name)+'/'+grade+'/'
    else:
      outfiledir = data_dir+ str(formatted_time)+'/'+str(school_name)+'/'+grade+'/'
    createFolder(outfiledir)
    outfilepath = os.path.join(outfiledir, name)
    #print(outfilepath)

    doc = SimpleDocTemplate(outfilepath, pagesize=letter,
                        rightMargin=72,leftMargin=72,
                        topMargin=72,bottomMargin=18)
    Story=[]

    if school_name == 'DCP Harper Woods' or school_name =='DCP Redford':
      logo = Image(links[school_name], width=2*inch, height=0.42*inch, kind='proportional')
    else:
      logo = Image(links[school_name])

    
    NumberOfTotalAbsences = str(row.NumberOfTotalAbsences)
    story = []
    #Set the logo 
    Story.append(logo)
     
    styles=getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
    ptext = '<font size=11>%s</font>' % formatted_time
     
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 12))
     
    ptext = '<font size=11>Dear Family of %s: </font>' % (full_name)
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    
    ptext = '<font size=11>We are writing to you because <u>our records show \
             that your child has missed more than 10 % of the school year\
              to date.</u> Please understand that all absences are considered \
              unexcused until proper documentation is presented in accordance \
              with the school handbook.  Regular attendance at school is critical\
               for a child’s educational growth and achievement, and we are \
               committed to your child’s success.  We look forward to partnering\
                with you to ensure that your child continues to have a great \
                year of learning and growth.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=11>If absences continue, we will schedule a time for you\
     to come to school for a meeting with our administration, with the goal being\
      to identify the factors contributing to your child’s absenteeism, discuss \
      ways we can improve your child’s attendance this year, and commit to a plan\
       of action together. These steps will be taken in accordance with the \
       attendance policy for Distinctive Schools.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
     
    ptext = '<font size=11>Thank you for your attention to this matter, and \
    thank you in advance for your partnership and a dedication to ensuring that \
    your child is set up for success in school this year. I look forward to \
    working with you to support your child in having a positive and productive year. </font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    ptext = '<font size=11>Sincerely,</font>'
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 24))
    ptext = '<font size=11>%s</font>'% (dnames[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=11>Director, %s</font>'% (school_name)
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=11>%s</font>'% (mails[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(PageBreak()) 


    #Second page barchart
    title_style = styles['Heading1']
    title_style.alignment = 1
    title = Paragraph("Absences Summary", title_style)
    Story.append(title)
    Story.append(Spacer(1, 24))
    pic = make_drawing(row)
    Story.append(pic)

    doc.build(Story)

In [43]:
df[df.NumberOfTotalAbsences==df['NumberOfTotalAbsences'].max()]

,NameOfInstitution,StudentUniqueId,GradeLevel,LastSurname,FirstName,NumberOfTotalAbsences,AverageAbsence
1,Plymouth Educational Center,20007170,K,Brown,AlayJia,63,6.1


In [44]:
df.shape

(174, 7)

# Running the code to generate the letter

In [0]:
#For English-Spanish letter
import time
start = time.clock() 
for i in range(df.shape[0]):
  build_a_letter(df.loc[i], False)
elapsed = time.clock()
elapsed = elapsed - start
print("Time spent in is: ", elapsed)

In [0]:
dcp = df[df.NameOfInstitution == 'DCP Harper Woods'|df.NameOfInstitution == 'DCP Redford']
for i in range(dcp.shape[0]):
  build_english_letter(dcp.loc[i], False)
elapsed = time.clock()
elapsed = elapsed - start
print("Time spent in is: ", elapsed)

In [45]:
#For English Letter only
import time
start = time.clock() 
for i in range(df.shape[0]):
  build_english_letter(df.loc[i], False)
elapsed = time.clock()
elapsed = elapsed - start
print("Time spent in is: ", elapsed)

Time spent in is:  4.887461


In [46]:
#Sanity check
!ls /content/gdrive/My\ Drive/Chronic_absenteeism_letter/

 2020-01-23		 'Chronic Absence - Absence List Import.csv'
 2020-02-07		 'Chronic Absence Letter #1 - Jan '\''20.gdoc'
 2020-02-13		 'Chronic Absence.xlsx'
 2.13ChronicAbsence.csv   Chronic_Absenteeism.ipynb


# Checking which way generate the letter faster

In [0]:
mni = df[150:]

In [0]:
import time
start = time.clock() 
for i in range(mni.shape[0]):
  build_a_letter(mni.loc[i], False)
elapsed = time.clock()
elapsed = elapsed - start
print("Time spent in is: ", elapsed)

,NameOfInstitution,StudentUniqueId,GradeLevel,LastSurname,FirstName,NumberOfTotalAbsences,AverageAbsence
150,Art in Motion,50197615,8,Bardney,Jazmynn,11,4.0
151,Art in Motion,50404026,8,Beard,Solene,10,4.0
152,Art in Motion,50301891,8,Boyd IV,Walter,11,4.0
153,Art in Motion,50089602,8,Brent,Robionte,12,4.0
154,Art in Motion,50242045,8,BURGES,DEMETRIA,10,4.0
155,Art in Motion,50726547,8,Burr,Kaylyn,11,4.0
156,Art in Motion,50254680,7,Davis,Alexandria,14,4.0
157,Art in Motion,60040857,7,Doxy,Xa'Ria,25,4.0
158,Art in Motion,50227895,8,Ford,Sariya,11,4.0
159,Art in Motion,50185861,8,Franklin,Emareona,24,4.0


In [0]:
import time
start = time.clock() 
mni.apply(lambda x: build_a_letter(x), axis=1)
elapsed = time.clock()
elapsed = elapsed - start
print("Time spent in is: ", elapsed)

Time spent in is:  0.8404169999999995
